# Differentiation in TensorFlow
In the presentation, we had an interesting discussion about how differentiation
works within TensorFlow. `tf.gradients` says it uses "symbolic differentiation,"
which is what I (Evan) said. Most computing-based differentiation strategies
do automatic or numerical differentiation, so the question is what TensorFlow *actually* uses.

The TensorFlow code (and documentation!) seems to suggest it's attempting to do 
symbolic differentiation (see
[the TensorFlow GitHub repo](https://github.com/tensorflow/tensorflow/blob/r1.3/tensorflow/python/ops/gradients_impl.py)
-- specifically looking for `symbolic_gradient`).

[Numerical differentiation](https://en.wikipedia.org/wiki/Numerical_differentiation)
is the process of adding small noise to the input of a function and monitoring
the change in the function. That doesn't seem to be anywhere in TensorFlow.

The definitions of symbolic and automatic differentiation seem to be somewhat blurry depending on where you look. [This blog](https://alexey.radul.name/ideas/2013/introduction-to-automatic-differentiation/) about automatic differentiation seems to suggest that symbolic differentiation is purely mechanical, with a set of known rules and little-to-no simplification. Automatic differentiation seems to be smarter.

TensorFlow seems more of a simple symbolic differentiation system, in that it has many rules for derivatives, that are applied via the chain rule. For example (from [this part of the codebase](https://github.com/tensorflow/tensorflow/blob/b474e55c23e5cc42b01a1ddea34751f01110deb6/tensorflow/python/ops/math_grad.py)):

```python
@ops.RegisterGradient("Abs")
def _AbsGrad(op, grad):
  x = op.inputs[0]
  return grad * math_ops.sign(x)
```

specifies that the derivative of the absolute value function is the
sign of the input (which would seem to answer the question about the $\ell1$ subgradient question). The `grad` term that is part of the input and multiplied
in the output is for backpropagation -- we start at the end and work
our way backward, updating the gradient as we go. By the time we reach
a node applying the absolute value function on `f(x)`, we've already computed
the chain rule for `f'(x)`.

As another example, `exp(x)` is implemented as
```python
@ops.RegisterGradient("Exp")
def _ExpGrad(op, grad):
  """Returns grad * exp(x)."""
  y = op.outputs[0]  # y = e^x
  with ops.control_dependencies([grad]):
    y = math_ops.conj(y)
    return grad * y
```
where the gradient of $\exp(x)$ is the output of the function times
the derivative of everything above this node in the graph: If we compute $z = 2 \exp(x)$, we can break this up into $y = \exp(x)$ and $z=2y$. Then $\frac{dz}{dy}=2$ and $\frac{dy}{dx}=y$. So, to compute the derivative $\frac{dz}{dx}$, we just do the chain rule (in this case, we're doing "backpropagation").

At all points, TensorFlow just does a lookup for each derivative based on rules predefined in the language, or defined in a custom operation. By default, new operations just compose existing operations, so the derivative is well-defined without additional work. But in some cases (or to improve efficiency based on leveraging some tricks) we can in principle define new fundamental operations. If you're interested in that, check out TensorFlow documentation on [creating an op](https://www.tensorflow.org/extend/adding_an_op). For almost all cases though, you shouldn't need to.

# Hacking symbolic differentiation
Because TensorFlow more-or-less does simple symbolic differentation, we can use the computation graph it constructs to explore the form of the derivative, rather than just the value for a particular input.

This is a very simple script that is incomplete -- it doesn't even handle matrix operations! -- but gives an idea of how gradients are computed.

First, construct the operation as $y = f(x[, z])$ for some function $f$. Then take the derivative (here, all taken with respect to $x$ -- the extra [0] is because tf.gradients returns a list always). That happens more or less symbolically.

Then, trace through the new graph created by that gradient, following along until we've reconstructed the full operation (we can print out an arbitrary operation using the getstr(tensor) function, but the gradient is what we're looking at here. Presumably, we could even actually write our own "TensorFlow interpreter" this way, but I'll leave that as an exercise for the reader.

This shows though that the gradient maintains symbolic structure, and really computes the chain rule just as you or I might. Then, whenever the value is needed, TensorFlow will compute the actual values using its definitions of elementary operations and functions (addition, subtraction, exp, log, etc.). 

In [23]:
import tensorflow as tf

# At least theoretically, the value of the constants are irrelevant.
# Use your own values if you'd like.
x = tf.constant(5.0, name="x")
z = tf.constant(2.0, name="z")

# Add any new variables here to print them out nicely.
variables_with_names = [x, z]




var_names = [t.name for t in variables_with_names]
sess = tf.Session()

def getstr(tensor):
    unary_ops = ["Exp", "Sign", "Abs", "Log", "Sin",
                 "Sinh", "Cos", "Cosh", "Sqrt", "Tan", "Tanh",
                 "Erf", "Erfc", "Acos", "Asin", "Atan", "Asinh",
                 "Acosh", "Atanh"]
    unary_replace_ops = {"Digamma": unichr(936),
                         "Lgamma": "ln " + unichr(915)}
    binary_replace_ops = {"Zeta": unichr(950),
                          "Maximum": "max",
                          "Minimum": "min"}

    # Already a variable we know about.
    if tensor.name in var_names:
        # TensorFlow takes a name "x", creates a unique number for
        # this session:
        # For example "x_11:0".
        try:
            idx = tensor.name.index("_")
            return tensor.name[0:idx]
        except:
            return tensor.name
    
    # Simple operations like Exp get dropped-in.
    elif tensor.op.type in unary_ops:
        return "%s(%s)" % (tensor.op.type.lower(),
                           getstr(tensor.op.inputs[0]))
    
    # Allow for closer-to-stats notation for certain functions
    # like the digamma.
    elif tensor.op.type in unary_replace_ops:
        return "%s(%s)" % (unary_replace_ops[tensor.op.type],
                           getstr(tensor.op.inputs[0]))
    
    # Same as unary_replace_ops, but for binary operations.
    elif tensor.op.type in binary_replace_ops:
        return "%s(%s, %s)" % (
            binary_replace_ops[tensor.op.type],
            getstr(tensor.op.inputs[0]),
            getstr(tensor.op.inputs[1]))
    
    # Handle some special-case operations
    elif tensor.op.type == "Square":
        return "(%s)^2" % getstr(tensor.op.inputs[0])
    elif tensor.op.type == "Pow":
        return "(%s)^(%s)" % (getstr(tensor.op.inputs[0]),
                              getstr(tensor.op.inputs[1]))
    
    # Multiplication, addition should use the infix operators
    # "+" and "*".
    elif tensor.op.type == "Mul":
        return "(%s * %s)" % (getstr(tensor.op.inputs[0]),
                              getstr(tensor.op.inputs[1]))
    elif tensor.op.type == "Sub":
        return "(%s - %s)" % (getstr(tensor.op.inputs[0]),
                              getstr(tensor.op.inputs[1]))
    elif tensor.op.type == "AddN":
        terms = [getstr(t) for t in tensor.op.inputs]
        return " + ".join(terms)
    elif tensor.op.type == "RealDiv":
        return "(%s / %s)" % (getstr(tensor.op.inputs[0]),
                              getstr(tensor.op.inputs[1]))
    
    # Don't need parens around a negation.
    elif tensor.op.type == "Neg":
        return "-%s" % getstr(tensor.op.inputs[0])
    
    # Const values (other than the variables in the computation)
    # are just constants we need to add/multiply, etc. Can
    # get their actual value in the graph.
    elif tensor.op.type == "Const":
        return "%s" % sess.run(tensor)
    
    # The rest of these are some odd tensorflow things.
    
    # Fill seems to happen on one side of a binary operator without
    # a proper value. Just enter a 1 instead.
    elif tensor.op.type == "Fill":
        return "1"
    # Sum is not like add, for some reason.
    elif tensor.op.type in ["Reshape", "Sum"]:
        return getstr(tensor.op.inputs[0])
    
    # If you want to extend this, here's a default to help you
    # know what type of operation is needed next.
    else:
        return tensor.op.type

y = tf.abs(x)
print(getstr(tf.gradients(y, x)[0]))
print

y = tf.square(x)
# Should match 2 x
print(getstr(tf.gradients(y, x)[0]))
print
    
y = tf.exp(x)
# Should match exp(x)
print(getstr(tf.gradients(y, x)[0]))
print

y = tf.exp(tf.square(x))
# should match 2 x exp(x^2)
print(getstr(tf.gradients(y, x)[0]))
print

y = x / tf.exp(tf.square(x))
print(getstr(tf.gradients(y, x)[0]))
print

y = tf.exp(tf.abs(x))
print(getstr(tf.gradients(y, x)[0]))
print

y = x * z
print(getstr(tf.gradients(y, x)[0]))

(1 * sign(x))

(1 * (2.0 * x))

(1 * exp(x))

((1 * exp((x)^2)) * (2.0 * x))

(1 / exp((x)^2)) + (((1 * ((-x / exp((x)^2)) / exp((x)^2))) * exp((x)^2)) * (2.0 * x))

((1 * exp(abs(x))) * sign(x))

(1 * z)
